In [4]:
import pandas as pd

def SMA(array, n):
    return pd.Series(array).rolling(n).mean()

def RSI(array, n):
    gain = pd.Series(array).diff()
    loss = gain.copy()
    gain[gain < 0] = 0
    loss[loss > 0] = 0
    rs = gain.ewm(n).mean() / loss.abs().ewm(n).mean() #ratio of average gain to average loss over n periods
    return 100 - 100 / (1 + rs)

**Buy condition**

- weekly RSI(30) ≥ daily RSI(30) > 70
- Close > MA(10) > MA(20) > MA(50) > MA(100)

**Close condition:**

- Daily close is more than 2% below MA(10)
- 8% fixed stop loss is hit


In [ ]:
from backtesting import Strategy, Backtest
from backtesting.lib import resample_apply


class System(Strategy):
    d_rsi = 30  # Daily RSI lookback periods
    w_rsi = 30  # Weekly
    level = 70
    
    def init(self):
        # Compute moving averages
        self.ma10 = self.I(SMA, self.data.Close, 10)
        self.ma20 = self.I(SMA, self.data.Close, 20)
        self.ma50 = self.I(SMA, self.data.Close, 50)
        self.ma100 = self.I(SMA, self.data.Close, 100)
        
        # Compute daily RSI(30)
        self.daily_rsi = self.I(RSI, self.data.Close, self.d_rsi)
        
        # To construct weekly RSI, use `resample_apply()`, a helper function from the backtesting library
        self.weekly_rsi = resample_apply(
            'W-FRI', RSI, self.data.Close, self.w_rsi) #uses friday's close data
        
        
    def next(self):
        price = self.data.Close[-1]
        
        # If we don't already have a position, and
        # if all conditions are satisfied, enter long.
        if (not self.position and
            self.daily_rsi[-1] > self.level and
            self.weekly_rsi[-1] > self.level and
            self.weekly_rsi[-1] > self.daily_rsi[-1] and
            self.ma10[-1] > self.ma20[-1] > self.ma50[-1] > self.ma100[-1] and
            price > self.ma10[-1]):
            
            # Buy at market price on next open, but do
            # set 8% fixed stop loss.
            self.buy(sl=.92 * price)
        
        # If the price closes 2% or more below 10-day MA
        # close the position, if any.
        elif price < .98 * self.ma10[-1]:
            self.position.close()
        

In [22]:
import yfinance as yf
EURUSD = yf.download("EURUSD=X", interval="5m", start="2025-03-01", end="2025-03-09")

if isinstance(EURUSD.columns, pd.MultiIndex):
    EURUSD.columns = EURUSD.columns.droplevel(1)  # Drop 'GOOG' from column names

if EURUSD.columns[0] == "Close":
    EURUSD = EURUSD[['Open', 'High', 'Low', 'Close', 'Volume']]

EURUSD.tail()

# backtest = Backtest(EURUSD, System, commission=0.002)
# stats = backtest.run()
# print(stats)


[*********************100%***********************]  1 of 1 completed


Price,Open,High,Low,Close,Volume
Datetime,,,,,
2025-03-07 22:05:00+00:00,1.083424,1.083424,1.083424,1.083424,0
2025-03-07 22:10:00+00:00,1.083424,1.083424,1.083424,1.083424,0
2025-03-07 22:15:00+00:00,1.083424,1.083424,1.083424,1.083424,0
2025-03-07 22:20:00+00:00,1.083424,1.083424,1.083424,1.083424,0
2025-03-07 22:25:00+00:00,1.083424,1.083424,1.083424,1.083424,0


In [18]:
import yfinance as yf
import pandas as pd

# Download Google stock data
GOOGLE = yf.download("GOOG", start="2018-01-01", end="2020-06-30", auto_adjust=True)



# Drop MultiIndex if necessary
if isinstance(GOOGLE.columns, pd.MultiIndex):
    GOOGLE.columns = GOOGLE.columns.droplevel(1)  # Drop 'GOOG' from column names

# Print current column order
print("Before reordering:\n", GOOGLE.head())

# Reorder columns if 'Close' is first
if GOOGLE.columns[0] == "Close":
    GOOGLE = GOOGLE[['Open', 'High', 'Low', 'Close', 'Volume']]  # Ensure correct order

# Print new column order
print("After reordering:\n", GOOGLE.head())

# Now it should work with Backtest()
from backtesting import Backtest

print("running backtest")
backtest = Backtest(GOOG, System, commission=0.002)
stats = backtest.run()
print(stats)


[*********************100%***********************]  1 of 1 completed

Before reordering:
 Price           Close       High        Low       Open    Volume
Date                                                            
2018-01-02  53.059353  53.156006  52.074393  52.229335  24752000
2018-01-03  53.930225  54.120042  52.970174  53.024977  28604000
2018-01-04  54.125523  54.482741  54.006052  54.205238  20092000
2018-01-05  54.914188  55.014829  54.404518  54.504163  25582000
2018-01-08  55.148846  55.364570  54.883799  54.914188  20952000
After reordering:
 Price            Open       High        Low      Close    Volume
Date                                                            
2018-01-02  52.229335  53.156006  52.074393  53.059353  24752000
2018-01-03  53.024977  54.120042  52.970174  53.930225  28604000
2018-01-04  54.205238  54.482741  54.006052  54.125523  20092000
2018-01-05  54.504163  55.014829  54.404518  54.914188  25582000
2018-01-08  54.914188  55.364570  54.883799  55.148846  20952000
running backtest
Start                     2004-08-